In [1]:
import pandas as pd 
import numpy as np 
import json 
import requests 
from ast import literal_eval
import geopy.distance

In [2]:
rental_df = pd.read_csv('../data/raw/All_Houses_Scraped.csv')

#drop links column 
rental_df = rental_df.drop('Unnamed: 0', axis = 1)

#drop entires without lat/lon 
#NOTE - There aren't any, just a precausionary step 
rental_df = rental_df.dropna(subset = ['coordinates'], axis = 0)

sample_rental_df = rental_df.iloc[0:100]

house_postcodes = [int(i[-1]) for i in sample_rental_df['name'].str.split()] 
sample_rental_df.loc[:,'house_postcode'] = house_postcodes

sample_rental_df.head(5)

/opt/anaconda3/envs/tf/lib/python3.7/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


,coordinates,cost_text,desc,name,rooms,house_postcode
0,"[-37.9032708, 145.0770553]",$680,Joseph Chatziconstantis,15 Fintonia Street Hughesdale VIC 3166,"['3 Beds', '2 Baths', '2 Parking']",3166
1,"[-37.9077745, 145.092738]",$650.00,James Drakopoulos,41 Bishop Street Oakleigh VIC 3166,"['3 Beds', '2 Baths', '2 Parking']",3166
2,"[-37.8922004, 145.0899378]",$500,Nigel Chee,109/6 Dalgety Street Oakleigh VIC 3166,"['1 Bed', '1 Bath', '1 Parking']",3166
3,"[-37.8972564, 145.0871089]",$570,Matthew Swinnerton,16/30 Swindon Road Oakleigh VIC 3166,"['3 Beds', '2 Baths', '2 Parking']",3166
4,"[-37.894307, 145.082706]",$475,Jason Gu,1/14 Bletchley Road Hughesdale VIC 3166,"['2 Beds', '1 Bath', '1 Parking']",3166


In [3]:
def cbd_distance(house_coords):
    
    melb_cbd = [-37.815207, 144.963937]
    
    journey = requests.get(f"http://router.project-osrm.org/route/v1/car/{house_coords[1]},{house_coords[0]};{melb_cbd[1]},{melb_cbd[0]}?overview=false""")
    
    distance_meters = json.loads(journey.content)['routes'][0]['legs'][0]['distance']
    
    return distance_meters / 1000

In [4]:
CBD_distances = []

for i in sample_rental_df['coordinates']:
    CBD_distances.append(cbd_distance(literal_eval(i)))
    
sample_rental_df.loc[:,'CBD_Distance'] = CBD_distances
sample_rental_df.head(5)

/opt/anaconda3/envs/tf/lib/python3.7/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


,coordinates,cost_text,desc,name,rooms,house_postcode,CBD_Distance
0,"[-37.9032708, 145.0770553]",$680,Joseph Chatziconstantis,15 Fintonia Street Hughesdale VIC 3166,"['3 Beds', '2 Baths', '2 Parking']",3166,18.9780
1,"[-37.9077745, 145.092738]",$650.00,James Drakopoulos,41 Bishop Street Oakleigh VIC 3166,"['3 Beds', '2 Baths', '2 Parking']",3166,19.0444
2,"[-37.8922004, 145.0899378]",$500,Nigel Chee,109/6 Dalgety Street Oakleigh VIC 3166,"['1 Bed', '1 Bath', '1 Parking']",3166,17.1238
3,"[-37.8972564, 145.0871089]",$570,Matthew Swinnerton,16/30 Swindon Road Oakleigh VIC 3166,"['3 Beds', '2 Baths', '2 Parking']",3166,17.3138
4,"[-37.894307, 145.082706]",$475,Jason Gu,1/14 Bletchley Road Hughesdale VIC 3166,"['2 Beds', '1 Bath', '1 Parking']",3166,17.5619


In [5]:
retained_cols = ['SCHOOL_NO', 'X' , 'Y', 'Postal_Postcode']
vic_schools = pd.read_csv('../data/raw/dv309_schoollocations2021.csv', encoding='cp1252',
                         usecols = retained_cols)

vic_schools['coordinates']= vic_schools[['Y','X']].values.tolist()

vic_schools.drop(['X', 'Y'], axis=1, inplace=True)

vic_schools = vic_schools.dropna(subset = ['coordinates', 'Postal_Postcode'], how = 'any').reset_index(drop = True)

In [6]:
def calculate_proximity(point1, point2):
    
    journey = requests.get(f"http://router.project-osrm.org/route/v1/car/{point1[1]},{point1[0]};{point2[1]},{point2[0]}?overview=false""")
    distance_meters = json.loads(journey.content)['routes'][0]['legs'][0]['distance']
    
    return distance_meters / 1000

In [7]:
def nearest_schools_distance(house):
    
    curr_house_postcode = house['house_postcode']
    
    curr_house_lat_lon = literal_eval(house['coordinates'])
    
    #vic_schools[vic_schools['Postal_Postcode'] == curr_house_postcode]
    
    curr_postcode_schools = vic_schools[vic_schools['Postal_Postcode'] == curr_house_postcode]
    
    nearest_schools_dist = []
    

    if len(curr_postcode_schools) >= 2:
        
        for i in curr_postcode_schools['coordinates']:
            
            dist_to_school = calculate_proximity(curr_house_lat_lon, i)
            nearest_schools_dist.append(dist_to_school)
            
        return sorted(nearest_schools_dist[0:2])
            
    else:
    
        lookup_range = [curr_house_postcode - 3, curr_house_postcode + 3]
    
        lookup_range_schools = vic_schools [(vic_schools ['Postal_Postcode'] >= lookup_range[0]) & 
                                        (vic_schools ['Postal_Postcode'] <= lookup_range[1])]
    
        print(len(lookup_range_schools))
        
        nearest_schools_dist = []
    

        if len(lookup_range_schools) > 0:

            for i in lookup_range_schools['coordinates']:

                dist_to_school = calculate_proximity(curr_house_lat_lon, i)
                nearest_schools_dist.append(dist_to_school)

        return sorted(nearest_schools_dist[0:2])
    
    return [np.nan, np.nan]

In [8]:
#two_nearest_schools = []

#for i in range(sample_rental_df.index[0], sample_rental_df.index[-1] + 1):
#    two_nearest_schools.append(nearest_schools_distance(sample_rental_df.iloc[i]))

In [9]:
nearest_school_1 = []
nearest_school_2 = []

for i in range(sample_rental_df.index[0], sample_rental_df.index[-1] + 1):
    
    two_nearest = nearest_schools_distance(sample_rental_df.iloc[i])
    nearest_school_1.append(two_nearest[0])
    nearest_school_2.append(two_nearest[1])
    


In [10]:
sample_rental_df['Nearest_school_1'] = nearest_school_1 
sample_rental_df['Nearest_school_2'] = nearest_school_2 

sample_rental_df.head(5)

/opt/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,coordinates,cost_text,desc,name,rooms,house_postcode,CBD_Distance,Nearest_school_1,Nearest_school_2
0,"[-37.9032708, 145.0770553]",$680,Joseph Chatziconstantis,15 Fintonia Street Hughesdale VIC 3166,"['3 Beds', '2 Baths', '2 Parking']",3166,18.9780,0.3622,2.0721
1,"[-37.9077745, 145.092738]",$650.00,James Drakopoulos,41 Bishop Street Oakleigh VIC 3166,"['3 Beds', '2 Baths', '2 Parking']",3166,19.0444,1.9066,2.1386
2,"[-37.8922004, 145.0899378]",$500,Nigel Chee,109/6 Dalgety Street Oakleigh VIC 3166,"['1 Bed', '1 Bath', '1 Parking']",3166,17.1238,0.4685,3.1575
3,"[-37.8972564, 145.0871089]",$570,Matthew Swinnerton,16/30 Swindon Road Oakleigh VIC 3166,"['3 Beds', '2 Baths', '2 Parking']",3166,17.3138,0.4079,2.4489
4,"[-37.894307, 145.082706]",$475,Jason Gu,1/14 Bletchley Road Hughesdale VIC 3166,"['2 Beds', '1 Bath', '1 Parking']",3166,17.5619,0.6560,2.1302


In [11]:
retained_columns = ['Hospital name','State','Latitude', 'Longitude', 'Postcode', 'Hospital_ID']

hospitals = pd.read_csv('../data/raw/myhospitals-contact-details.csv', encoding='cp1252',
                       usecols = retained_columns)

hospitals = hospitals[hospitals['State'] == 'Vic'].reset_index(drop = True)

hospitals['coordinates']= hospitals[['Latitude','Longitude']].values.tolist()
hospitals = hospitals.drop(['Latitude','Longitude'], axis = 1)

hospitals.head()

,Hospital name,Postcode,State,Hospital_ID,coordinates
0,Albert Road Clinic,3004,Vic,8,"[-37.834246, 144.972151]"
1,Albury Wodonga Health [Wodonga Campus],3690,Vic,10,"[-36.131791, 146.879417]"
2,Alexandra District Hospital,3714,Vic,12,"[-37.195065, 145.716143]"
3,Alpine Health [Bright],3741,Vic,16,"[-36.732701, 146.965924]"
4,Alpine Health [Mount Beauty],3699,Vic,17,"[-36.743354, 147.169359]"


In [12]:
def nearest_hospital(house):
    
    curr_house_postcode = house['house_postcode']
    curr_house_lat_lon = literal_eval(house['coordinates'])
    curr_postcode_hospital = hospitals[hospitals['Postcode'] == curr_house_postcode].reset_index(drop = True)
    
   
    
    if len(curr_postcode_hospital) == 1:
        
        dist_to_hosp = calculate_proximity(curr_house_lat_lon, curr_postcode_hospital.iloc[0]['coordinates'])
        
        return dist_to_hosp
        
    if len(curr_postcode_hospital) >= 2:
        
        nearest_hosp_dist = []
        
        for i in curr_postcode_hospital['coordinates']:
            
            dist_to_hosp = calculate_proximity(curr_house_lat_lon, i)
            nearest_hosp_dist.append(dist_to_hosp)
            
        return sorted(nearest_hosp_dist)[0]

    
    found = 0 
    i = 1 
    
    lookup_range = [curr_house_postcode - i, curr_house_postcode + i]
    lookup_range_hosp = hospitals [(hospitals ['Postcode'] >= lookup_range[0]) & 
                                        (hospitals ['Postcode'] <= lookup_range[1])].reset_index(drop = True)
    
    while(found != 1):
        
        if len(lookup_range_hosp) == 0:
            i = i + 1
            
        else:
            found = 1 
            
    if len(lookup_range_hosp) == 1:
        
        dist_to_hosp = calculate_proximity(curr_house_lat_lon, lookup_range_hosp.iloc[0]['coordinates'])
        
        return dist_to_hosp
    
    
    
    if len(lookup_range_hosp) > 1:
        
        nearest_hosp_dist = list()
        
        for i in lookup_range_hosp['coordinates']:
            
            dist_to_hosp = calculate_proximity(curr_house_lat_lon, i)
            nearest_hosp_dist.append(dist_to_hosp)
            
        return sorted(nearest_hosp_dist)[0]
    
    return np.nan

In [13]:
nearest_hospitals = []

for i in sample_rental_df.index:
    
    nearest_hosp = nearest_hospital(sample_rental_df.iloc[i])
    nearest_hospitals.append(nearest_hosp)

In [16]:
sample_rental_df['nearest_hospital'] = nearest_hospitals
sample_rental_df.head(5)

/opt/anaconda3/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,coordinates,cost_text,desc,name,rooms,house_postcode,CBD_Distance,Nearest_school_1,Nearest_school_2,nearest_hospital
0,"[-37.9032708, 145.0770553]",$680,Joseph Chatziconstantis,15 Fintonia Street Hughesdale VIC 3166,"['3 Beds', '2 Baths', '2 Parking']",3166,18.9780,0.3622,2.0721,3.6709
1,"[-37.9077745, 145.092738]",$650.00,James Drakopoulos,41 Bishop Street Oakleigh VIC 3166,"['3 Beds', '2 Baths', '2 Parking']",3166,19.0444,1.9066,2.1386,4.4447
2,"[-37.8922004, 145.0899378]",$500,Nigel Chee,109/6 Dalgety Street Oakleigh VIC 3166,"['1 Bed', '1 Bath', '1 Parking']",3166,17.1238,0.4685,3.1575,5.7190
3,"[-37.8972564, 145.0871089]",$570,Matthew Swinnerton,16/30 Swindon Road Oakleigh VIC 3166,"['3 Beds', '2 Baths', '2 Parking']",3166,17.3138,0.4079,2.4489,5.0104
4,"[-37.894307, 145.082706]",$475,Jason Gu,1/14 Bletchley Road Hughesdale VIC 3166,"['2 Beds', '1 Bath', '1 Parking']",3166,17.5619,0.6560,2.1302,5.1099
